In [23]:
from binance.client import Client
import pytz
import datetime
import dateutil.parser
import pandas as pd
import smtplib 
from stockstats import StockDataFrame
import datetime as dt
from IPython.display import clear_output
import time
import numpy as np
import matplotlib.pyplot as py
import urllib3
from binance.client import Client
import plotly.graph_objects as go
import pandas as pd

In [24]:
def convert_datetime_timezone(dt, tz1, tz2):
    tz1 = pytz.timezone(tz1)
    tz2 = pytz.timezone(tz2)
    dt = datetime.datetime.strptime(dt,'%Y-%m-%d %H:%M:%S')
    dt = tz1.localize(dt)
    dt = dt.astimezone(tz2)
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    return dt

In [25]:
def scrap_binance_price(unit,interval):

    api_key = 'LKa422RMjzx2tsXESIoSQ5qJzqi7LaC7iDFf9qdSrYjhJrZ6a8CcvOdniJWtfX13'
    api_secret = 'EvomPthpqwgTjeuqYFWBQM9HtE52Zeq6SuKrZSROqhd8g2tvbz086zVZqbbJGxsZ'

    client = Client(api_key, api_secret, {"verify": False, "timeout": 20})
    klines = client.get_historical_klines("BTCUSDT", unit, interval)

    data = pd.DataFrame();
    tradingdate = []
    openprice = []
    highprice = []
    lowprice = []
    closeprice = [] 
    for i in range(len(klines)):   
        time = datetime.datetime.utcfromtimestamp(klines[i][0]/1000).strftime('%Y-%m-%d %H:%M:%S')
        local_time = convert_datetime_timezone(time,'GMT','Australia/Adelaide')
        format_time = dateutil.parser.parse(local_time)
        tradingdate.append(format_time)
        openprice.append(float(klines[i][1]))
        highprice.append(float(klines[i][2]))
        lowprice.append(float(klines[i][3]))
        closeprice.append(float(klines[i][4]))
    data['datetime'] = tradingdate
    data['low'] = lowprice
    data['high'] = highprice
    data['open'] = openprice
    data['close'] = closeprice
    return data

In [26]:
def heikin_ashi(df):
    
    heikin_ashi_df = pd.DataFrame(index=df.index.values, columns=['open', 'high', 'low', 'close'])
    
    heikin_ashi_df['close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    
    for i in range(len(df)):
        if i == 0:
            heikin_ashi_df.iat[0, 0] = df['open'].iloc[0]
        else:
            heikin_ashi_df.iat[i, 0] = (heikin_ashi_df.iat[i-1, 0] + heikin_ashi_df.iat[i-1, 3]) / 2
        
    heikin_ashi_df['high'] = heikin_ashi_df.loc[:, ['open', 'close']].join(df['high']).max(axis=1)
    
    heikin_ashi_df['low'] = heikin_ashi_df.loc[:, ['open', 'close']].join(df['low']).min(axis=1)
    
    heikin_ashi_df['datetime'] = df['datetime']
    return heikin_ashi_df

In [27]:
def add_indicator(df1,df2):
    #1: buy indicator
    #w: sell indicator
    #0: no activity
    point = []
    #MACD_EMA_SHORT: default to 12
    #MACD_EMA_LONG: default to 26
    #MACD_EMA_SIGNAL: default to 9
    
    #df1: original price dataset
    #df2: heikin-ashi price dataset
    
    
    df = StockDataFrame.retype(df1)
    df['macd'] = df.get('macd') # calculate MACD
    #1: buy indicator
    #-1: sell indicator
    #0: no activity
    MACD = df
    point = []
    previous = 0
    current = 0
    #MACD_EMA_SHORT: default to 12
    #MACD_EMA_LONG: default to 26
    #MACD_EMA_SIGNAL: default to 9
    point.append(0)
    hasBitcoin = False
    for i in range(1,len(MACD)):
        previous = MACD['macdh'][i-1];
        current = MACD['macdh'][i];
        current_open = df2['open'][i]
        current_close = df2['close'][i];
        if(previous<0 and current >0 and current_close>current_open and hasBitcoin == False):
            point.append(1)
            hasBitcoin = True
        elif(previous>0 and current <0 and current_close<current_open and hasBitcoin == True):
            point.append(2)
            hasBitcoin = False

        else:
            point.append(0)
    MACD['indicator'] = point
    return MACD

In [28]:
def round_down(value, decimals):
    factor = 1 / (10 ** decimals)
    return (value // factor) * factor

In [29]:
def email_notification(message):
    info = pd.read_csv('C:\\Users\\Ray\\Desktop\\E_Info.csv')
    gmailaddress = info['username'][0]
    gmailpassword = info['password'][0]
    mailToOne = info['addressOne'][0]
    mailToTwo = info['addressTwo'][0]
    msg = "Buy the Bitcoin"
    mailServer = smtplib.SMTP('smtp.gmail.com' , 587)
    mailServer.starttls()
    mailServer.login(gmailaddress , gmailpassword)
    #mailServer.sendmail(gmailaddress, mailToOne , message)
    #mailServer.sendmail(gmailaddress, mailToTwo , message)
    mailServer.quit()

In [30]:
def trading(treasure,test,money,bitcoin,bnb):
    
    record = {}
    initial_money = money
    initial_bitcoin = bitcoin
    initial_bnb = bnb
    #client.order_market_buy(symbol = 'BTCUSDT',quantity = round_down(initial_bnb,6))
    A = []
    B = []
    C = []
    D = []
    E = []
    F = []
    G = []
    
    commission_rate = 0.00075
    
    test.reset_index(drop = True, inplace = True)
    comission_state = True
    balance = 100
    index = 0
    for i in range (len(test)):
        #set the current transfer index    
        index += 1
        #set the transaction rule
        #set the rule of buy bitcoin
        if(test['indicator'][i] == 1 and money > 0 and comission_state == True and bnb>0):
            if(money/test['close'][i]<balance):
                if(i<len(test)-1):
                    bitcoin_exchange = round_down(money/test['close'][i],8)
                    bnb -= money/18*commission_rate
                    balance -= bitcoin_exchange
                    bitcoin += bitcoin_exchange
                    money -= bitcoin_exchange*test['close'][i]
                    record[test['datetime'][i]] = 1
                else:
                    bitcoin_exchange = round_down(money/test['close'][i],8)
                    bnb -= money/18*commission_rate
                    balance -= bitcoin_exchange
                    bitcoin += bitcoin_exchange
                    money -= bitcoin_exchange*test['close'][i]
                    if(record.get(test['datetime'][i]) != 1):
                        record[test['datetime'][i]] = 1
                        email_notification("Buy the Bitcoin Right Now! -- 1 Hour Economic Model.")
                    #client.order_market_buy(symbol = 'BTCUSDT',quantity = round_down(bitcoin_exchange,8))
            else:
                bitcoin += balance
                money -= balance*test['close'][i]
                bnb -= money/18*commission_rate
                comission_state = False
                record[test['datetime'][i]] = 1
                if(i<len(test)-1):
                    if(record.get(test['datetime'][i]) != 1):
                        record[test['datetime'][i]] = 1
                        email_notification("Buy the Bitcoin Right Now! -- 1 Hour Economic Model.")
                        #client.order_market_buy(symbol = 'BTCUSDT',quantity = round_down(balance,8) )
        #set the rule of sell bitcoin
        elif(test['indicator'][i] == 2 and bitcoin>0 and bnb>0):
            if(i<len(test)-1):
                #each extraction opertion the system would charge commission fee
                bnb -= bitcoin*test['close'][i]/18*commission_rate
                money += bitcoin*test['close'][i]
                bitcoin = 0
                record[test['datetime'][i]] = 2
            else:              
                #each extraction opertion the system would charge commission fee
                bnb -= bitcoin*test['close'][i]/18*commission_rate
                money += bitcoin*test['close'][i]
                bitcoin = 0
                if(record.get(test['datetime'][i]) != 2):
                    record[test['datetime'][i]] = 2
                    email_notification("Sell the Bitcoin Right Now! -- 1 Hour Economic Model.")
                #client.order_market_sell(symbol = 'BTCUSDT',quantity = round_down(bitcoin,8))
        #the system would run 24h daily 
        if(index>=24*4):
            index = 0
            balance = 100
            comission_state = True
            
        A.append(test['datetime'][i]);
        B.append(money)
        C.append(bitcoin)
        D.append(bitcoin*test['close'][i]+money)
        E.append(test['indicator'][i])
        F.append(i+1)
        G.append(test['close'][i])
 
    treasure['datetime'] = A
    treasure['index'] = F
    treasure['money'] = B
    treasure['bitcoin'] = C
    treasure['total'] = D
    treasure['indicator'] = E
    treasure['close'] = G
    
    initial_assert = initial_money+initial_bitcoin*test['close'][0]+initial_bnb*18
    TotalAssert = bitcoin*test['close'][len(test)-1]+money+bnb*18
    TransactionDate = test['datetime'][len(test)-1]-test['datetime'][0]
    print("Initial money is:" + str(initial_money))
    print("Initial bitcoin is:" + str(initial_bitcoin))
    print("Initial bnb is:" + str(initial_bnb))
    print("Investment Start Time:" + str(test['datetime'][0]) )
    print("Investment End Time:" + str(test['datetime'][len(test)-1]) )
    print("Investment Periods: "+str(TransactionDate.days) )
    print("Current Money:" + str(money) )
    print("Current Bitcoin: "+ str(bitcoin) )
    print("Current BNB: "+ str(bnb) )
    print("Return on Investment is:"+ str( (TotalAssert-initial_assert)/initial_assert*100)+"%")
    print("Current Total Assert is: "+ str(TotalAssert) )
    if(TotalAssert/initial_assert<0.9):
        print("Trading curb occured")
        money += bitcoin * test['close'][len(test)-1]*commission_rate
        bnb -= bitcoin*test['close'][len(test)-1]*commission_rate
        bitcoin = 0
        alarm = True
    return treasure   

In [31]:
def display_candles(data):

    df = data

    fig = go.Figure(data=[go.Candlestick(x=df['datetime'],
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'])])
    fig.show()

In [32]:
def run_bot(time_interval,time_interval_name,time_range):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    pd.set_option('display.float_format',lambda x : '%.8f' % x)

    api_key = 'LKa422RMjzx2tsXESIoSQ5qJzqi7LaC7iDFf9qdSrYjhJrZ6a8CcvOdniJWtfX13'
    api_secret = 'EvomPthpqwgTjeuqYFWBQM9HtE52Zeq6SuKrZSROqhd8g2tvbz086zVZqbbJGxsZ'

    client = Client(api_key, api_secret, {"verify": False, "timeout": 20})
    money = 0
    bitcoin = 0.04612
    bnb = 1
    alarm = False
    initial_data = scrap_binance_price(time_interval,time_range)
    data = initial_data[:len(initial_data)-1]
    #loop
    code = 0
    while(bnb>0):
        #try:
        latest_data = scrap_binance_price(time_interval, time_interval_name+" ago UTC")
        if(data['datetime'][len(data)-1] == latest_data['datetime'][0]):
            data.drop(data.index[len(data)-1],inplace = True)
        data = pd.concat([data, latest_data], axis=0, join='outer', ignore_index=False, keys=None,
                  levels=None, names=None, verify_integrity=False, copy=True)
        data.reset_index(drop = True, inplace = True)
        heikin_ashi_data = data.copy()
        heikin_ashi_info = heikin_ashi(heikin_ashi_data)
        candle_sticks = data.copy()
        test = add_indicator(candle_sticks,heikin_ashi_info)
        treasure = pd.DataFrame()
        trading_record = trading(treasure,test,money,bitcoin,bnb)

        x_positive = []
        y_positive = []
        yp_positive = []
        x_negative = []
        y_negative = []  
        yp_negative = []
        for i in range(len(treasure)):
            if(treasure['indicator'][i] == 1):
                x_positive.append(treasure['datetime'][i])
                y_positive.append(treasure['total'][i])
                yp_positive.append(treasure['close'][i])
            elif(treasure['indicator'][i] == 2):
                x_negative.append(treasure['datetime'][i])
                y_negative.append(treasure['total'][i])
                yp_negative.append(treasure['close'][i])

        #assert growth graph
        fig_assert = go.Figure()
        x = treasure['datetime']
        y = treasure['total']

        
        fig_assert.add_trace(go.Scatter(x=x,y=y,line=dict(color='skyblue', width=2),mode='lines',name='close price'))
        fig_assert.add_trace(go.Scatter(x = x_positive,y=y_positive,line=dict(color='green',width=1),mode='markers', name='buy point'))
        fig_assert.add_trace(go.Scatter(x = x_negative,y=y_negative,line=dict(color='orange',width=1),mode='markers', name='sell point'))
        fig_assert.show()
        

        #price flowing graph
        
        fig_price = go.Figure()
        
        xp = treasure['datetime']
        yp = treasure['close']
        
        fig_price.add_trace(go.Scatter(x=xp,y=yp,line=dict(color='skyblue', width=2),mode='lines',name='close price'))
        fig_price.add_trace(go.Scatter(x = x_positive,y=yp_positive,line=dict(color='green',width=1),mode='markers', name='buy point'))
        fig_price.add_trace(go.Scatter(x = x_negative,y=yp_negative,line=dict(color='orange',width=1),mode='markers', name='sell point'))
        fig_price.show()

        code+=1
        display_candles(heikin_ashi_info)
        break
        if(alarm == True):
            email_notification("Sell the Bitcoin Right Now! -- 1 Hour Economic Model.")
            break
        #except:
        #    print("Something went wrong")
        time.sleep(30)
        clear_output()
        
    return trading_record

In [33]:
trading_record = run_bot('4h','4 hours','2 months ago UTC')
trading_record

NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values


Initial money is:0
Initial bitcoin is:0.04612
Initial bnb is:1
Investment Start Time:2019-10-09 14:30:00
Investment End Time:2019-12-09 10:30:00
Investment Periods: 60
Current Money:381.2233525369003
Current Bitcoin: 0
Current BNB: 0.6251423767246205
Return on Investment is:-0.7001352630788995%
Current Total Assert is: 392.4759153179435


,datetime,index,money,bitcoin,total,indicator,close
0,2019-10-09 14:30:00,1,0.00000000,0.04612000,377.24315200,0,8179.60000000
1,2019-10-09 18:30:00,2,0.00000000,0.04612000,377.53370800,0,8185.90000000
2,2019-10-09 22:30:00,3,0.00000000,0.04612000,394.32600000,0,8550.00000000
3,2019-10-10 02:30:00,4,0.00000000,0.04612000,395.80091760,0,8581.98000000
4,2019-10-10 06:30:00,5,0.00000000,0.04612000,394.82132880,0,8560.74000000
5,2019-10-10 10:30:00,6,0.00000000,0.04612000,394.02668120,0,8543.51000000
6,2019-10-10 14:30:00,7,0.00000000,0.04612000,397.26522760,0,8613.73000000
7,2019-10-10 18:30:00,8,0.00000000,0.04612000,393.14071600,0,8524.30000000
8,2019-10-10 22:30:00,9,0.00000000,0.04612000,393.89662280,0,8540.69000000
9,2019-10-11 02:30:00,10,0.00000000,0.04612000,394.51647560,0,8554.13000000


In [20]:
trading_record = run_bot('4h','4 hour','6 months ago UTC')
trading_record

NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values


Initial money is:0
Initial bitcoin is:0.1
Initial bnb is:5
Investment Start Time:2019-06-09 13:30:00
Investment End Time:2019-12-09 10:30:00
Investment Periods: 182
Current Money:4.5193599589765654e-05
Current Bitcoin: 0.1414681
Current BNB: 2.570416874058746
Return on Investment is:26.187711626837235%
Current Total Assert is: 1110.3269364816572


,datetime,index,money,bitcoin,total,indicator,close
0,2019-06-09 13:30:00,1,0.00000000,0.10000000,789.90100000,0,7899.01000000
1,2019-06-09 17:30:00,2,0.00000000,0.10000000,785.26300000,0,7852.63000000
2,2019-06-09 21:30:00,3,0.00000000,0.10000000,771.67200000,0,7716.72000000
3,2019-06-10 01:30:00,4,0.00000000,0.10000000,773.20600000,0,7732.06000000
4,2019-06-10 05:30:00,5,0.00000000,0.10000000,762.81300000,0,7628.13000000
5,2019-06-10 09:30:00,6,0.00000000,0.10000000,765.44600000,0,7654.46000000
6,2019-06-10 13:30:00,7,0.00000000,0.10000000,769.87100000,0,7698.71000000
7,2019-06-10 17:30:00,8,0.00000000,0.10000000,790.23600000,1,7902.36000000
8,2019-06-10 21:30:00,9,0.00000000,0.10000000,791.44500000,0,7914.45000000
9,2019-06-11 01:30:00,10,0.00000000,0.10000000,794.58100000,0,7945.81000000


In [14]:
trading_record = run_bot('2h','2 hours','7 days ago UTC')
trading_record

NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values


Initial money is:0
Initial bitcoin is:0.1
Initial bnb is:5
Investment Start Time:2019-12-02 12:30:00
Investment End Time:2019-12-09 10:30:00
Investment Periods: 6
Current Money:682.6126118773
Current Bitcoin: 0
Current BNB: 4.621269819612561
Return on Investment is:-7.275662365377462%
Current Total Assert is: 765.795468630326


,datetime,index,money,bitcoin,total,indicator,close
0,2019-12-02 12:30:00,1,0.00000000,0.10000000,735.88400000,0,7358.84000000
1,2019-12-02 14:30:00,2,0.00000000,0.10000000,728.10200000,0,7281.02000000
2,2019-12-02 16:30:00,3,0.00000000,0.10000000,725.29600000,0,7252.96000000
3,2019-12-02 18:30:00,4,0.00000000,0.10000000,729.82400000,0,7298.24000000
4,2019-12-02 20:30:00,5,0.00000000,0.10000000,729.79100000,1,7297.91000000
5,2019-12-02 22:30:00,6,0.00000000,0.10000000,730.71300000,0,7307.13000000
6,2019-12-03 00:30:00,7,0.00000000,0.10000000,730.87600000,0,7308.76000000
7,2019-12-03 02:30:00,8,0.00000000,0.10000000,728.59900000,0,7285.99000000
8,2019-12-03 04:30:00,9,727.39800000,0.00000000,727.39800000,2,7273.98000000
9,2019-12-03 06:30:00,10,0.00005585,0.09959853,727.39800000,1,7303.30000000
